In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, SAGEConv
from torch_geometric.data import Data
from torch_geometric.nn import global_mean_pool
import torch_scatter

import numpy as np
from sklearn.metrics import ndcg_score
from collections import defaultdict
import pandas as pd

from pathlib import Path

In [2]:
project_root = Path("C://Users//DELL//Desktop//the-year-25-26//scalable-graph-based-movie-recommender")

In [3]:
ratings = pd.read_csv(project_root / 'data' / 'processed' / 'ratings_gnn.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,1225734739
1,1,110,4.0,1225865086
2,1,158,4.0,1225733503
3,1,260,4.5,1225735204
4,1,356,5.0,1225735119


In [4]:
movies = pd.read_csv(project_root / 'data' / 'processed' / 'movies_gnn.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
tags = pd.read_csv(project_root / 'data' / 'processed' / 'tags_gnn.csv')
tags.head()

,userId,movieId,tag,timestamp
0,302,3863,atmospheric,1476691609
1,302,3863,beautiful cinematography,1476691614
2,302,3863,stylized,1476691602
3,302,4226,great ending,1476691644
4,302,4226,psychological,1476691635


In [6]:
genome_scores= pd.read_csv(project_root / 'data' / 'processed' / 'genome_scores_gnn.csv')
genome_scores.head()

,movieId,tagId,relevance
0,1,1,0.03200
1,1,2,0.02225
2,1,3,0.07000
3,1,4,0.05900
4,1,5,0.12300


In [7]:
genome_tags= pd.read_csv(project_root / 'data' / 'processed' / 'genome_tags_gnn.csv')
genome_tags.head()

,tagId,tag
0,1,007
1,2,007 (series)
2,3,18th century
3,4,1920s
4,5,1930s


In [8]:
user_ids = ratings['userId'].unique()
movie_ids = ratings['movieId'].unique()
num_users = len(user_ids)
num_movies = len(movie_ids)

# Map original IDs to continuous indices
user_mapping = {id: idx for idx, id in enumerate(user_ids)}
movie_mapping = {id: idx for idx, id in enumerate(movie_ids)}

ratings['user_idx'] = ratings['userId'].map(user_mapping)
ratings['movie_idx'] = ratings['movieId'].map(movie_mapping)

In [9]:
# Filter content data
tags_filtered = tags[tags['movieId'].isin(movie_ids)]
genome_scores_filtered = genome_scores[genome_scores['movieId'].isin(movie_ids)]

# Create content features (reduce genome dimensionality)
from scipy.sparse import csc_matrix
genome_matrix = csc_matrix(
    (genome_scores_filtered['relevance'].values,
     (genome_scores_filtered['movieId'].map(movie_mapping).values,
      genome_scores_filtered['tagId'].values)),
    shape=(num_movies, genome_tags['tagId'].max() + 1)
).toarray()

genome_matrix

array([[0.     , 0.032  , 0.02225, ..., 0.033  , 0.077  , 0.01825],
       [0.     , 0.0525 , 0.031  , ..., 0.08975, 0.0895 , 0.0235 ],
       [0.     , 0.03275, 0.04125, ..., 0.008  , 0.10025, 0.01475],
       ...,
       [0.     , 0.02975, 0.03075, ..., 0.0065 , 0.106  , 0.0165 ],
       [0.     , 0.03125, 0.03425, ..., 0.00575, 0.087  , 0.01175],
       [0.     , 0.04125, 0.044  , ..., 0.00625, 0.1225 , 0.01775]],
      shape=(5979, 1129))

In [10]:
# Reduce genome features to 64 dimensions
from sklearn.decomposition import PCA
pca = PCA(n_components=64)
genome_features_reduced = pca.fit_transform(genome_matrix)
movie_content_features = torch.tensor(genome_features_reduced, dtype=torch.float)
movie_content_features

tensor([[ 7.9723e-01,  1.0766e+00,  3.0914e+00,  ..., -3.2717e-02,
          4.9399e-02, -4.4391e-01],
        [ 3.4215e+00,  2.6226e+00,  3.4195e+00,  ..., -7.1543e-02,
          1.1128e-01, -2.6499e-01],
        [-2.0368e+00, -5.4134e-01,  1.1547e+00,  ...,  3.4155e-01,
         -7.3131e-03,  1.0712e-01],
        ...,
        [ 2.4176e-01, -1.2308e+00, -5.9882e-01,  ...,  5.6452e-02,
         -5.8670e-02, -8.3411e-02],
        [-1.8002e+00, -9.2147e-01,  1.5571e-02,  ..., -1.3413e-01,
          1.8907e-03, -8.8710e-02],
        [-1.3067e+00, -4.0330e-01, -9.0030e-01,  ...,  1.6591e-01,
         -8.4876e-02, -1.8701e-01]])

In [11]:
# Create features for both users and movies with same dimension
base_features = torch.randn(num_users + num_movies, 64)
user_features = base_features[:num_users]  # 64-dim users
movie_features = base_features[num_users:] + 0.1 * movie_content_features  # 64-dim movies with content bias

# Combine features (same dimension for both)
x = torch.cat([user_features, movie_features], dim=0)  # [num_users + num_movies, 64]
x

tensor([[-0.2530,  0.1174,  0.8302,  ..., -0.5667, -0.1320, -1.1239],
        [-0.8906,  0.2787,  0.4533,  ...,  0.3281, -0.0848, -1.5943],
        [ 0.5214,  0.4572, -1.2895,  ..., -0.9175,  0.3495, -0.5758],
        ...,
        [-0.4667, -0.0059, -0.1572,  ..., -0.9803,  1.1263, -0.7283],
        [ 1.1180,  1.3720,  0.2362,  ...,  0.4294, -1.0032, -0.0793],
        [-0.7784, -0.9990,  0.9899,  ...,  1.4126,  1.9528, -1.5964]])

In [12]:
# Create a bipartite graph
user_idx = ratings['user_idx'].values
movie_idx = ratings['movie_idx'].values + num_users

edge_index_np = np.column_stack([
    np.concatenate([user_idx, movie_idx]),
    np.concatenate([movie_idx, user_idx])
]).T

edge_index = torch.from_numpy(edge_index_np)
ratings_tensor = torch.tensor(ratings['rating'].values, dtype=torch.float)
edge_attr = torch.cat([ratings_tensor, ratings_tensor])

from torch_geometric.data import Data
data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr)
print(data)

Data(x=[20979, 64], edge_index=[2, 5219794], edge_attr=[5219794])


In [13]:

from torch.utils.data import Dataset, DataLoader
import os

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from scipy.optimize import minimize
import torch_scatter
from torch_geometric.nn import GCNConv
import warnings
warnings.filterwarnings('ignore')

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cpu


In [15]:

# MATRIX FACTORIZATION MODEL

class MatrixFactorization(nn.Module):
    def __init__(self, num_users, num_movies, embed_dim=64, dropout_rate=0.1):
        super().__init__()
        self.user_embed = nn.Embedding(num_users, embed_dim)
        self.movie_embed = nn.Embedding(num_movies, embed_dim)
        self.dropout = nn.Dropout(dropout_rate)
        nn.init.normal_(self.user_embed.weight, std=0.1)
        nn.init.normal_(self.movie_embed.weight, std=0.1)
        
    def forward(self, user_idx, movie_idx):
        user_vec = self.dropout(self.user_embed(user_idx)) 
        movie_vec = self.dropout(self.movie_embed(movie_idx))
        return (user_vec * movie_vec).sum(dim=1)

In [16]:

# LIGHTGCN MODEL

class LightGCN_Rating(nn.Module):
    def __init__(self, num_users, num_movies, embed_dim=64, num_layers=2):
        super().__init__()
        self.num_users = num_users
        self.num_movies = num_movies
        self.embed_dim = embed_dim
        self.num_layers = num_layers
        
        # User and movie embeddings
        self.user_embed = nn.Embedding(num_users, embed_dim)
        self.movie_embed = nn.Embedding(num_movies, embed_dim)
        
        # Learnable layer weights (LightGCN paper)
        self.layer_weights = nn.Parameter(torch.ones(num_layers + 1) / (num_layers + 1))
        
        # Rating prediction head
        self.rating_predictor = nn.Sequential(
            nn.Linear(embed_dim * 2, 128),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )
        
        # Initialize embeddings
        nn.init.xavier_uniform_(self.user_embed.weight)
        nn.init.xavier_uniform_(self.movie_embed.weight)
        
    def forward(self, edge_index, user_idx, movie_idx):
        # Get initial embeddings
        user_embs = self.user_embed.weight
        movie_embs = self.movie_embed.weight
        all_embs = torch.cat([user_embs, movie_embs], dim=0)
        
        # Store embeddings for each layer
        embs_list = [all_embs]
        
        # Propagate through graph layers
        for _ in range(self.num_layers):
            all_embs = self.propagate(edge_index, all_embs)
            embs_list.append(all_embs)
        
        # Weighted sum of embeddings across layers (LightGCN style)
        final_embs = torch.stack(embs_list, dim=0)
        weighted_embs = torch.sum(
            self.layer_weights.unsqueeze(1).unsqueeze(2) * final_embs, 
            dim=0
        )
        
        # Get user and movie representations
        user_repr = weighted_embs[user_idx]
        movie_repr = weighted_embs[movie_idx + self.num_users]
        
        # Concatenate and predict rating
        rating_input = torch.cat([user_repr, movie_repr], dim=1)
        rating = self.rating_predictor(rating_input).squeeze()
        
        # Scale rating to [1, 5] range
        rating = torch.sigmoid(rating) * 4 + 1
        
        return rating
    
    def propagate(self, edge_index, x):
        row, col = edge_index
        deg = torch_scatter.scatter_add(torch.ones_like(row), row, dim=0, dim_size=x.size(0))
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]
        out = torch_scatter.scatter_add(x[col] * norm.unsqueeze(1), row, dim=0, dim_size=x.size(0))
        return out

In [17]:

# GRAPHREC MODEL

class GraphRec(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim=64, dropout=0.3):
        super().__init__()
        # Project input features
        self.input_projection = (torch.nn.Linear(input_dim, hidden_dim) 
                                if input_dim != hidden_dim 
                                else torch.nn.Identity())
        
        # Graph convolutional layers
        self.conv1 = GCNConv(hidden_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        
        # Batch normalization
        self.bn1 = torch.nn.BatchNorm1d(hidden_dim)
        self.bn2 = torch.nn.BatchNorm1d(hidden_dim)
        
        # Dropout
        self.dropout = torch.nn.Dropout(dropout)
        
        # Rating predictor
        self.rating_predictor = torch.nn.Sequential(
            torch.nn.Linear(hidden_dim * 2, 64),
            torch.nn.BatchNorm1d(64),
            torch.nn.ReLU(),
            torch.nn.Dropout(dropout),
            
            torch.nn.Linear(64, 32),
            torch.nn.BatchNorm1d(32),
            torch.nn.ReLU(),
            torch.nn.Dropout(dropout),
            
            torch.nn.Linear(32, 16),
            torch.nn.ReLU(),
            torch.nn.Dropout(dropout/2),
            
            torch.nn.Linear(16, 1)
        )
        
        self._init_weights()
    
    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, torch.nn.Linear):
                torch.nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    torch.nn.init.zeros_(m.bias)
    
    def forward(self, x, edge_index, user_idx, movie_idx):
        # Project input features
        h = self.input_projection(x)
        
        # Apply GNN layers
        h = self.conv1(h, edge_index)
        h = self.bn1(h)
        h = F.relu(h)
        h = self.dropout(h)
        
        h = self.conv2(h, edge_index)
        h = self.bn2(h)
        h = F.relu(h)
        h = self.dropout(h)
        
        # Get user and movie representations
        user_repr = h[user_idx]
        movie_repr = h[movie_idx]
        
        # Predict rating
        rating_input = torch.cat([user_repr, movie_repr], dim=1)
        return self.rating_predictor(rating_input).squeeze()

print("✓ Model architectures defined")

✓ Model architectures defined


In [18]:
# Load preprocessed TRAINING data (graph structure built from training set)
preprocessed_data_path = '../models/preprocessed_data.pt'
preprocessed_data = torch.load(preprocessed_data_path, weights_only=False)

data = preprocessed_data['data']
user_mapping = preprocessed_data['user_mapping']  # user_id -> index (from TRAINING)
movie_mapping = preprocessed_data['movie_mapping']  # movie_id -> index (from TRAINING)

# This is the TRAINING graph structure
x = data.x.to(device)  # Node features from TRAINING
edge_index = data.edge_index.to(device)  # Graph edges from TRAINING interactions

num_users = len(user_mapping)
num_movies = len(movie_mapping)

print(f"✓ TRAINING graph loaded")
print(f"  Users: {num_users:,}")
print(f"  Movies: {num_movies:,}")
print(f"  Edges: {edge_index.shape[1]:,}")
print(f"  Node features: {x.shape}")
print(f"\nThis graph will be used to compute embeddings for VALIDATION predictions")

✓ TRAINING graph loaded
  Users: 15,000
  Movies: 5,979
  Edges: 5,219,794
  Node features: torch.Size([20979, 64])

This graph will be used to compute embeddings for VALIDATION predictions


In [19]:
print("Loading Matrix Factorization model...")

# Load checkpoint
mf_checkpoint = torch.load('../models/best_mf_model.pth', weights_only=False)

# Initialize model
mf_model = MatrixFactorization(
    num_users=num_users,
    num_movies=num_movies,
    embed_dim=64,
    dropout_rate=0.2  # Match your training config
)

# Load the state dict directly since checkpoint is just the state_dict
mf_model.load_state_dict(mf_checkpoint)
mf_model = mf_model.to(device)
mf_model.eval()

print(f"✓ MF model loaded")
print(f"  Parameters: {sum(p.numel() for p in mf_model.parameters()):,}")

Loading Matrix Factorization model...
✓ MF model loaded
  Parameters: 1,342,656


In [20]:
# First, check the LightGCN checkpoint structure
lgcn_checkpoint = torch.load('../models/best_rating_lightgcn_model.pth', weights_only=False)
print("Keys in LightGCN checkpoint:", list(lgcn_checkpoint.keys()))
print("Type of checkpoint:", type(lgcn_checkpoint))

# Initialize model
lgcn_model = LightGCN_Rating(
    num_users=num_users,
    num_movies=num_movies,
    embed_dim=64,
    num_layers=3  # Match your training config
)

# Load based on the structure
lgcn_checkpoint = torch.load('../models/best_rating_lightgcn_model.pth', weights_only=False)
lgcn_model.load_state_dict(lgcn_checkpoint)
lgcn_model = lgcn_model.to(device)
lgcn_model.eval()

print(f"✓ LightGCN model loaded")
print(f"  Parameters: {sum(p.numel() for p in lgcn_model.parameters()):,}")

Keys in LightGCN checkpoint: ['layer_weights', 'user_embed.weight', 'movie_embed.weight', 'rating_predictor.0.weight', 'rating_predictor.0.bias', 'rating_predictor.3.weight', 'rating_predictor.3.bias', 'rating_predictor.5.weight', 'rating_predictor.5.bias']
Type of checkpoint: <class 'collections.OrderedDict'>
✓ LightGCN model loaded
  Parameters: 1,367,493


In [21]:
# First, check the GraphRec checkpoint structure
gr_checkpoint = torch.load('../models/best_graphrec_model.pth', weights_only=False)
print("Keys in GraphRec checkpoint:", list(gr_checkpoint.keys()))
print("Type of checkpoint:", type(gr_checkpoint))

# Initialize model
gr_model = GraphRec(
    input_dim=64,  # Match your training config
    hidden_dim=64,
    dropout=0.3
)

# Load based on the structure
gr_checkpoint = torch.load('../models/best_graphrec_model.pth', weights_only=False)
gr_model.load_state_dict(gr_checkpoint)
gr_model = gr_model.to(device)
gr_model.eval()

print(f"✓ GraphRec model loaded")
print(f"  Parameters: {sum(p.numel() for p in gr_model.parameters()):,}")


print("ALL MODELS LOADED SUCCESSFULLY")


Keys in GraphRec checkpoint: ['conv1.bias', 'conv1.lin.weight', 'conv2.bias', 'conv2.lin.weight', 'bn1.weight', 'bn1.bias', 'bn1.running_mean', 'bn1.running_var', 'bn1.num_batches_tracked', 'bn2.weight', 'bn2.bias', 'bn2.running_mean', 'bn2.running_var', 'bn2.num_batches_tracked', 'rating_predictor.0.weight', 'rating_predictor.0.bias', 'rating_predictor.1.weight', 'rating_predictor.1.bias', 'rating_predictor.1.running_mean', 'rating_predictor.1.running_var', 'rating_predictor.1.num_batches_tracked', 'rating_predictor.4.weight', 'rating_predictor.4.bias', 'rating_predictor.5.weight', 'rating_predictor.5.bias', 'rating_predictor.5.running_mean', 'rating_predictor.5.running_var', 'rating_predictor.5.num_batches_tracked', 'rating_predictor.8.weight', 'rating_predictor.8.bias', 'rating_predictor.11.weight', 'rating_predictor.11.bias']
Type of checkpoint: <class 'collections.OrderedDict'>
✓ GraphRec model loaded
  Parameters: 19,649
ALL MODELS LOADED SUCCESSFULLY


In [22]:
# Load VALIDATION data (these are user-item pairs NOT in training)
validation_df = pd.read_csv('../data/external/val_split.csv')

print(f"Original VALIDATION set size: {len(validation_df):,}")
print("These are unseen user-item pairs we want to predict ratings for.\n")

# Filter to only users and items that EXIST in the TRAINING graph
valid_users = validation_df['userId'].isin(user_mapping.keys())
valid_movies = validation_df['movieId'].isin(movie_mapping.keys())

common_val_df = validation_df[valid_users & valid_movies].copy()

print(f"Common VALIDATION set size: {len(common_val_df):,}")
print(f"Filtered out: {len(validation_df) - len(common_val_df):,} pairs")
print("(Filtered pairs contain users/items NOT in training graph)")

# Map validation user/item IDs to TRAINING indices
common_val_df['user_idx'] = common_val_df['userId'].map(user_mapping)
common_val_df['movie_idx'] = common_val_df['movieId'].map(movie_mapping)

# Verify all mappings succeeded
assert common_val_df['user_idx'].notna().all(), "Some user indices are missing"
assert common_val_df['movie_idx'].notna().all(), "Some movie indices are missing"

print("\n✓ Common validation set created successfully!")
print("\nSample validation pairs:")
print(common_val_df[['userId', 'movieId', 'rating', 'user_idx', 'movie_idx']].head())

Original VALIDATION set size: 260,990
These are unseen user-item pairs we want to predict ratings for.

Common VALIDATION set size: 260,990
Filtered out: 0 pairs
(Filtered pairs contain users/items NOT in training graph)

✓ Common validation set created successfully!

Sample validation pairs:
   userId  movieId  rating  user_idx  movie_idx
0  141084    56367     4.5      6357       1296
1  324282     1385     4.5     14675       1715
2   31676     1193     3.5      1336        147
3  233914     1945     5.0     10539       1582
4  155789      942     3.5      6992       4065


In [23]:

print("GENERATING PREDICTIONS ON VALIDATION SET")

print("\n1. Matrix Factorization predictions...")

# These are VALIDATION user-item pairs (not in training)
user_indices = torch.tensor(common_val_df['user_idx'].values, 
                            dtype=torch.long, device=device)
item_indices = torch.tensor(common_val_df['movie_idx'].values, 
                            dtype=torch.long, device=device)

with torch.no_grad():
    # MF predicts ratings for VALIDATION pairs using learned embeddings
    mf_predictions = mf_model(user_indices, item_indices).cpu().numpy()

# Calculate RMSE on VALIDATION set
true_ratings = common_val_df['rating'].values
mf_rmse = np.sqrt(mean_squared_error(true_ratings, mf_predictions))
mf_mae = np.mean(np.abs(true_ratings - mf_predictions))

print(f"   RMSE: {mf_rmse:.4f}")
print(f"   MAE:  {mf_mae:.4f}")
print(f"   Predictions shape: {mf_predictions.shape}")

GENERATING PREDICTIONS ON VALIDATION SET

1. Matrix Factorization predictions...
   RMSE: 0.7818
   MAE:  0.5950
   Predictions shape: (260990,)


In [24]:
print("\n2. LightGCN predictions...")

user_indices = torch.tensor(common_val_df['user_idx'].values, 
                            dtype=torch.long, device=device)
item_indices = torch.tensor(common_val_df['movie_idx'].values, 
                            dtype=torch.long, device=device)

with torch.no_grad():
    # LightGCN uses TRAINING edge_index to compute embeddings
    # Then predicts ratings for VALIDATION user-item pairs
    lgcn_predictions = lgcn_model(edge_index, user_indices, item_indices).cpu().numpy()

# Calculate RMSE on VALIDATION set
lgcn_rmse = np.sqrt(mean_squared_error(true_ratings, lgcn_predictions))
lgcn_mae = np.mean(np.abs(true_ratings - lgcn_predictions))

print(f"   RMSE: {lgcn_rmse:.4f}")
print(f"   MAE:  {lgcn_mae:.4f}")
print(f"   Predictions shape: {lgcn_predictions.shape}")
print(f"   Note: Used TRAINING edge_index for graph convolutions")


2. LightGCN predictions...
   RMSE: 0.8102
   MAE:  0.6141
   Predictions shape: (260990,)
   Note: Used TRAINING edge_index for graph convolutions


In [25]:
print("\n3. GraphRec predictions...")

user_indices = torch.tensor(common_val_df['user_idx'].values, 
                            dtype=torch.long, device=device)

# GraphRec expects movie indices OFFSET by num_users
movie_indices = torch.tensor(common_val_df['movie_idx'].values + num_users, 
                             dtype=torch.long, device=device)

with torch.no_grad():
    # GraphRec uses TRAINING x and edge_index to compute embeddings
    # Then predicts ratings for VALIDATION user-item pairs
    gr_predictions = gr_model(x, edge_index, user_indices, movie_indices).cpu().numpy()

# Calculate RMSE on VALIDATION set
gr_rmse = np.sqrt(mean_squared_error(true_ratings, gr_predictions))
gr_mae = np.mean(np.abs(true_ratings - gr_predictions))

print(f"   RMSE: {gr_rmse:.4f}")
print(f"   MAE:  {gr_mae:.4f}")
print(f"   Predictions shape: {gr_predictions.shape}")
print(f"   Note: Used TRAINING x and edge_index for GCN layers")


3. GraphRec predictions...
   RMSE: 1.3058
   MAE:  1.0937
   Predictions shape: (260990,)
   Note: Used TRAINING x and edge_index for GCN layers


In [26]:
print("INDIVIDUAL MODEL PERFORMANCE ON VALIDATION SET")

results_df = pd.DataFrame({
    'Model': ['Matrix Factorization', 'LightGCN', 'GraphRec'],
    'RMSE': [mf_rmse, lgcn_rmse, gr_rmse],
    'MAE': [mf_mae, lgcn_mae, gr_mae]
})

print("\n" + results_df.to_string(index=False))
print(f"\n✓ Best single model: {results_df.loc[results_df['RMSE'].idxmin(), 'Model']}")
print(f"✓ Best RMSE: {results_df['RMSE'].min():.4f}")
print(f"\nAll predictions made on {len(common_val_df):,} VALIDATION pairs")

INDIVIDUAL MODEL PERFORMANCE ON VALIDATION SET

               Model     RMSE      MAE
Matrix Factorization 0.781759 0.594995
            LightGCN 0.810176 0.614109
            GraphRec 1.305842 1.093661

✓ Best single model: Matrix Factorization
✓ Best RMSE: 0.7818

All predictions made on 260,990 VALIDATION pairs


In [27]:
# Save all predictions made on VALIDATION set
predictions_dict = {
    'mf_predictions': mf_predictions,
    'lightgcn_predictions': lgcn_predictions,
    'graphrec_predictions': gr_predictions,
    'true_ratings': true_ratings,
    'validation_data': common_val_df[['userId', 'movieId', 'rating', 
                                      'user_idx', 'movie_idx']].copy(),
    'info': {
        'description': 'Predictions on VALIDATION set using models trained on TRAINING set',
        'num_validation_samples': len(common_val_df),
        'num_training_users': num_users,
        'num_training_movies': num_movies,
        'training_edges': edge_index.shape[1]
    }
}

output_path = '../predictions/ensemble_predictions.pt'
torch.save(predictions_dict, output_path)
print(f"✓ VALIDATION predictions saved to {output_path}")

✓ VALIDATION predictions saved to ../predictions/ensemble_predictions.pt


In [28]:

print("OPTIMIZING ENSEMBLE WEIGHTS ON VALIDATION SET")

# Objective function: minimize RMSE on VALIDATION set
def ensemble_rmse(weights):
    w_mf, w_lgcn, w_gr = weights
    ensemble_preds = (w_mf * mf_predictions + 
                     w_lgcn * lgcn_predictions + 
                     w_gr * gr_predictions)
    return np.sqrt(mean_squared_error(true_ratings, ensemble_preds))

# Constraints: weights sum to 1 and are non-negative
constraints = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}
bounds = [(0, 1), (0, 1), (0, 1)]

# Initial guess: equal weights
initial_weights = [1/3, 1/3, 1/3]

print("Optimizing weights...")
# Optimize
result = minimize(
    ensemble_rmse,
    initial_weights,
    method='SLSQP',
    bounds=bounds,
    constraints=constraints
)

optimal_weights = result.x
optimal_rmse = result.fun

print("\n✓ Optimization Complete!")
print(f"\nOptimal weights:")
print(f"  MF:       {optimal_weights[0]:.4f}")
print(f"  LightGCN: {optimal_weights[1]:.4f}")
print(f"  GraphRec: {optimal_weights[2]:.4f}")
print(f"\nEnsemble RMSE on VALIDATION: {optimal_rmse:.4f}")

# Calculate improvement
best_single_rmse = min(mf_rmse, lgcn_rmse, gr_rmse)
improvement = best_single_rmse - optimal_rmse
improvement_pct = (improvement / best_single_rmse) * 100

print(f"\nBest single model RMSE: {best_single_rmse:.4f}")
print(f"Ensemble improvement: {improvement:.4f} ({improvement_pct:.2f}%)")

if improvement > 0:
    print(f"✓ Ensemble is better by {improvement:.4f} RMSE points!")
else:
    print(f"⚠ Ensemble not better than best single model")

OPTIMIZING ENSEMBLE WEIGHTS ON VALIDATION SET
Optimizing weights...

✓ Optimization Complete!

Optimal weights:
  MF:       0.9460
  LightGCN: 0.0540
  GraphRec: 0.0000

Ensemble RMSE on VALIDATION: 0.7817

Best single model RMSE: 0.7818
Ensemble improvement: 0.0001 (0.01%)
✓ Ensemble is better by 0.0001 RMSE points!


In [29]:
import numpy as np
correlation_matrix = np.corrcoef([mf_predictions, lgcn_predictions, gr_predictions])
print("Prediction correlations:")
print("MF vs LightGCN:", correlation_matrix[0,1])
print("MF vs GraphRec:", correlation_matrix[0,2])
print("LightGCN vs GraphRec:", correlation_matrix[1,2])

Prediction correlations:
MF vs LightGCN: 0.9421318200336354
MF vs GraphRec: 0.16254633358231846
LightGCN vs GraphRec: 0.1659962180258239


In [30]:
# Save optimal weights for future use
weights_dict = {
    'optimal_weights': optimal_weights,
    'ensemble_rmse': optimal_rmse,
    'individual_rmses': {
        'MF': mf_rmse,
        'LightGCN': lgcn_rmse,
        'GraphRec': gr_rmse
    },
    'individual_maes': {
        'MF': mf_mae,
        'LightGCN': lgcn_mae,
        'GraphRec': gr_mae
    },
    'info': {
        'optimized_on': 'validation_set',
        'num_samples': len(common_val_df)
    }
}

weights_path = '../models/ensemble_weights.pt'
torch.save(weights_dict, weights_path)
print(f"✓ Optimal weights saved to {weights_path}")

✓ Optimal weights saved to ../models/ensemble_weights.pt


In [31]:
def ensemble_predict(user_ids, movie_ids, optimal_weights):
    """
    Generate ensemble predictions for NEW user-movie pairs.
    
    Args:
        user_ids: List of original user IDs (must be in training)
        movie_ids: List of original movie IDs (must be in training)
        optimal_weights: Array of [w_mf, w_lgcn, w_gr]
    
    Returns:
        ensemble_preds: Array of ensemble predictions
        individual_preds: Dict with predictions from each model
    """
    # Verify all IDs exist in training
    for uid in user_ids:
        if uid not in user_mapping:
            raise ValueError(f"User ID {uid} not in training data")
    for mid in movie_ids:
        if mid not in movie_mapping:
            raise ValueError(f"Movie ID {mid} not in training data")
    
    # Map to TRAINING indices
    user_indices_list = [user_mapping[uid] for uid in user_ids]
    movie_indices_list = [movie_mapping[mid] for mid in movie_ids]
    
    user_tensor = torch.tensor(user_indices_list, dtype=torch.long, device=device)
    movie_tensor = torch.tensor(movie_indices_list, dtype=torch.long, device=device)
    
    # Get predictions from each model using TRAINING graph
    with torch.no_grad():
        # MF predictions
        mf_preds = mf_model(user_tensor, movie_tensor).cpu().numpy()
        
        # LightGCN predictions (uses TRAINING edge_index)
        lgcn_preds = lgcn_model(edge_index, user_tensor, movie_tensor).cpu().numpy()
        
        # GraphRec predictions (uses TRAINING x and edge_index)
        movie_tensor_gr = torch.tensor(
            [idx + num_users for idx in movie_indices_list],
            dtype=torch.long, 
            device=device
        )
        gr_preds = gr_model(x, edge_index, user_tensor, movie_tensor_gr).cpu().numpy()
    
    # Combine predictions using optimal weights
    w_mf, w_lgcn, w_gr = optimal_weights
    ensemble_preds = w_mf * mf_preds + w_lgcn * lgcn_preds + w_gr * gr_preds
    
    return ensemble_preds, {
        'mf': mf_preds,
        'lightgcn': lgcn_preds,
        'graphrec': gr_preds
    }

print("✓ Ensemble prediction function defined!")
print("\nThis function can predict on ANY new pairs")
print("(as long as users/items exist in the TRAINING graph)")

✓ Ensemble prediction function defined!

This function can predict on ANY new pairs
(as long as users/items exist in the TRAINING graph)


In [32]:
print("TESTING ENSEMBLE ON SAMPLE VALIDATION PAIRS")


# Take 5 random validation pairs
sample_indices = np.random.choice(len(common_val_df), size=5, replace=False)
test_user_ids = common_val_df['userId'].iloc[sample_indices].tolist()
test_movie_ids = common_val_df['movieId'].iloc[sample_indices].tolist()
test_true_ratings = common_val_df['rating'].iloc[sample_indices].tolist()

print(f"\nTesting on 5 random validation pairs...")

ensemble_preds, individual_preds = ensemble_predict(
    test_user_ids, 
    test_movie_ids, 
    optimal_weights
)

print("\nDetailed Predictions:")

for i in range(len(test_user_ids)):
    true_rating = test_true_ratings[i]
    print(f"\nUser {test_user_ids[i]}, Movie {test_movie_ids[i]} → True: {true_rating:.1f}")
    print(f"  MF:       {individual_preds['mf'][i]:.3f} (error: {abs(true_rating - individual_preds['mf'][i]):.3f})")
    print(f"  LightGCN: {individual_preds['lightgcn'][i]:.3f} (error: {abs(true_rating - individual_preds['lightgcn'][i]):.3f})")
    print(f"  GraphRec: {individual_preds['graphrec'][i]:.3f} (error: {abs(true_rating - individual_preds['graphrec'][i]):.3f})")
    print(f"  Ensemble: {ensemble_preds[i]:.3f} (error: {abs(true_rating - ensemble_preds[i]):.3f}) ★")

# Calculate mean absolute error for this sample
sample_mae = np.mean([abs(test_true_ratings[i] - ensemble_preds[i]) for i in range(len(test_user_ids))])
print(f"\nSample MAE: {sample_mae:.4f}")

TESTING ENSEMBLE ON SAMPLE VALIDATION PAIRS

Testing on 5 random validation pairs...

Detailed Predictions:

User 164541, Movie 2706 → True: 4.0
  MF:       3.191 (error: 0.809)
  LightGCN: 3.313 (error: 0.687)
  GraphRec: 2.517 (error: 1.483)
  Ensemble: 3.197 (error: 0.803) ★

User 330707, Movie 2011 → True: 3.5
  MF:       3.715 (error: 0.215)
  LightGCN: 3.678 (error: 0.178)
  GraphRec: 2.441 (error: 1.059)
  Ensemble: 3.713 (error: 0.213) ★

User 296688, Movie 7285 → True: 2.5
  MF:       3.531 (error: 1.031)
  LightGCN: 3.617 (error: 1.117)
  GraphRec: 3.266 (error: 0.766)
  Ensemble: 3.535 (error: 1.035) ★

User 218572, Movie 274971 → True: 3.5
  MF:       3.354 (error: 0.146)
  LightGCN: 3.390 (error: 0.110)
  GraphRec: 3.147 (error: 0.353)
  Ensemble: 3.356 (error: 0.144) ★

User 305823, Movie 49647 → True: 2.0
  MF:       2.206 (error: 0.206)
  LightGCN: 1.926 (error: 0.074)
  GraphRec: 3.315 (error: 1.315)
  Ensemble: 2.191 (error: 0.191) ★

Sample MAE: 0.4773
